In [18]:
#NN library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np
import torch
from typing import Optional, Union, Tuple


import torch.nn.functional as F
from torch.distributions import Distribution
from torch.distributions import Bernoulli, Normal, StudentT, Poisson, NegativeBinomial

from torch.distributions import constraints


from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATSx, NHITS
from neuralforecast.tsdataset import TimeSeriesDataset
from neuralforecast.utils import AirPassengers, AirPassengersPanel, AirPassengersStatic
from neuralforecast.losses.numpy import rmse, mape
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
from neuralforecast.losses.pytorch import MSE


In [19]:
from numpy.random import seed
from random import randrange

In [20]:
#Realized volatility 
def Yang_Zhang_RV_yahoo(tickers, start=None, end=None, period=None, interval=None):
    # importing needed libraries
    import yfinance as yf
    import pandas as pd
    import numpy as np
    import warnings
    warnings.filterwarnings("ignore")
    
    #Data extraction
    if period==None:
        data=yf.download(tickers=tickers, start=start, end=end, interval=interval)
    else:
        data=yf.download(tickers=tickers, period=period, interval=interval)
    #dropping N/A values
    if data.isnull().values.any()==True:
        data=data.dropna()
        print("Rows with missing values were removed")
    else:
        data=data
    data['Date_only']=data.index.strftime('%Y-%m-%d')
    data['Date_only']=pd.to_datetime(data['Date_only'])
    #Sorting function
    def replace_first_price(group):
      # Set a new value to nan
      new_oi = np.nan 
      group.iloc[0, group.columns.get_loc('oi')] = new_oi
      return group    
    # Yang_Zhang_RV formula is give as:
    # RV^2 = Vo + k*Vc + (1-k)*Vrs
    # where Vo = 1/(n-1)*sum(Oi-Obar)^2
    # with oi = normalized opening price at time t and Obar = mean of normalized opening prices
    # Vc = = 1/(n-1)*sum(ci-Cbar)^2
    # with ci = normalized close price at time t and Cbar = mean of normalized close prices
    # k = 0.34/(1.34+(n+1)/(n-1))
    # with n = total number of days or time periods considered
    # Vrs (Rogers & Satchell RV proxy) = ui(ui-ci)+di(di-ci)
    # with ui = ln(Hi/Oi), ci = ln(Ci/Oi), di=(Li/Oi), oi = ln(Oi/Ci-1)
    # where Hi = high price at time t and Li = low price at time t
    
    data["ui"]=np.log(np.divide(data["High"][1:],data["Open"][1:]))
    data["ci"]=np.log(np.divide(data["Close"][1:],data["Open"][1:]))
    data["di"]=np.log(np.divide(data["Low"][1:],data["Open"][1:]))
    data["oi"]=np.log(np.divide(data["Open"][1:],data["Close"][:len(data)-1].shift(1)))
    data = data.groupby('Date_only').apply(replace_first_price)
    data=data[1:]
    data["RS"]=data["ui"]*(data["ui"]-data["ci"])+data["di"]*(data["di"]-data["ci"])
    RS_var= data["RS"].groupby(pd.Grouper(freq='W')).mean().dropna()
    Vc_and_Vo=data[["oi", "ci"]].groupby(pd.Grouper(freq='W')).var().dropna()
    n=int(len(data)/len(RS_var))
    k = 0.34/(1.34+(n+1)/(n-1))
    Yang_Zhang_RV=np.sqrt((1-k)*RS_var+Vc_and_Vo["oi"]+Vc_and_Vo["ci"]*k)
    Yang_Zhang_RV_df=pd.DataFrame(Yang_Zhang_RV)
    Yang_Zhang_RV_df.rename(columns={0: "Yang & Zhang RV proxy"},inplace=True)
    
    return Yang_Zhang_RV_df

def Yang_Zhang_RV_own_data(data):
    # importing needed libraries
    import pandas as pd
    import numpy as np
    import warnings
    warnings.filterwarnings("ignore")
    #sorting first values of each day
    data['Date_only']=data.index.strftime('%Y-%m-%d')
    data['Date_only']=pd.to_datetime(data['Date_only'])
    #Sorting function
    def replace_first_price(group):
      # Set a new value to nan
      new_oi = np.nan 
      group.iloc[0, group.columns.get_loc('oi')] = new_oi
      return group
    # Yang_Zhang_RV formula is give as:
    # RV^2 = Vo + k*Vc + (1-k)*Vrs
    # where Vo = 1/(n-1)*sum(Oi-Obar)^2
    # with oi = normalized opening price at time t and Obar = mean of normalized opening prices
    # Vc = = 1/(n-1)*sum(ci-Cbar)^2
    # with ci = normalized close price at time t and Cbar = mean of normalized close prices
    # k = 0.34/(1.34+(n+1)/(n-1))
    # with n = total number of days or time periods considered
    # Vrs (Rogers & Satchell RV proxy) = ui(ui-ci)+di(di-ci)
    # with ui = ln(Hi/Oi), ci = ln(Ci/Oi), di=(Li/Oi), oi = ln(Oi/Ci-1)
    # where Hi = high price at time t and Li = low price at time t
    
    data["ui"]=np.log(np.divide(data["High"][1:],data["Open"][1:]))
    data["ci"]=np.log(np.divide(data["Close"][1:],data["Open"][1:]))
    data["di"]=np.log(np.divide(data["Low"][1:],data["Open"][1:]))
    data["oi"]=np.log(np.divide(data["Open"][1:],data["Close"][:len(data)-1].shift(1)))
    #Apply sort function
    data = data.groupby('Date_only').apply(replace_first_price)
    data=data[1:]
    data["RS"]=data["ui"]*(data["ui"]-data["ci"])+data["di"]*(data["di"]-data["ci"])
    RS_var= data["RS"].groupby(pd.Grouper(freq='D')).mean().dropna()
    Vc_and_Vo=data[["oi", "ci"]].groupby(pd.Grouper(freq='D')).var().dropna()
    n=int(len(data)/len(RS_var))
    k = 0.34/(1.34+(n+1)/(n-1))
    Yang_Zhang_RV=np.sqrt((1-k)*RS_var+Vc_and_Vo["oi"]+Vc_and_Vo["ci"]*k)
    Yang_Zhang_RV_df=pd.DataFrame(Yang_Zhang_RV)
    Yang_Zhang_RV_df.rename(columns={0: "Yang & Zhang RV proxy"},inplace=True)
    
    return Yang_Zhang_RV_df
    
    
    
def Multivariate_Yang_Zhang_RV_own_data(data_list):
    Multivariate_Yang_Zhang_RV=[]
    for i in range(len(data_list)):
        Yang_Zhang_RV_df=Yang_Zhang_RV_own_data(data=data_list[i])
        Multivariate_Yang_Zhang_RV.append(Yang_Zhang_RV_df)
    return Multivariate_Yang_Zhang_RV
    
    
def Multivariate_Yang_Zhang_RV_yahoo(tickers, start=None, end=None, period=None, interval=None):
    # importing needed libraries
    import yfinance as yf
    import pandas as pd
    import numpy as np
    import warnings
    warnings.filterwarnings("ignore")
    
    #Data extraction
    if period==None:
        data=yf.download(tickers=tickers, start=start, end=end, interval=interval)
    else:
        data=yf.download(tickers=tickers, period=period, interval=interval)
    #dropping N/A values
    if data.isnull().values.any()==True:
        data=data.dropna()
        print("Rows with missing values were removed")
    else:
        data=data

    data=data.unstack().reset_index(name="Actuals").rename(columns={"level_1":"Stocks"}).set_index("Datetime").pivot(columns=['Stocks','level_0'])
    data=data['Actuals']
    data['Date_only']=data.index.strftime('%Y-%m-%d')
    data['Date_only']=pd.to_datetime(data['Date_only'])
    #Sorting function
    def replace_first_price(group):
      # Set a new value to nan
      new_oi = np.nan 
      group.iloc[0, group.columns.get_loc('oi')] = new_oi
      return group    
    # Yang_Zhang_RV formula is give as:
    # RV^2 = Vo + k*Vc + (1-k)*Vrs
    # where Vo = 1/(n-1)*sum(Oi-Obar)^2
    # with oi = normalized opening price at time t and Obar = mean of normalized opening prices
    # Vc = = 1/(n-1)*sum(ci-Cbar)^2
    # with ci = normalized close price at time t and Cbar = mean of normalized close prices
    # k = 0.34/(1.34+(n+1)/(n-1))
    # with n = total number of days or time periods considered
    # Vrs (Rogers & Satchell RV proxy) = ui(ui-ci)+di(di-ci)
    # with ui = ln(Hi/Oi), ci = ln(Ci/Oi), di=(Li/Oi), oi = ln(Oi/Ci-1)
    # where Hi = high price at time t and Li = low price at time t
    
    Multivariate_Yang_Zhang_RV=[]
    for i in range(len(tickers)):
        data1=data[tickers[i]]
        data1["ui"]=np.log(np.divide(data1["High"][1:],data1["Open"][1:]))
        data1["ci"]=np.log(np.divide(data1["Close"][1:],data1["Open"][1:]))
        data1["di"]=np.log(np.divide(data1["Low"][1:],data1["Open"][1:]))
        data1["oi"]=np.log(np.divide(data1["Open"][1:],data1["Close"][:len(data1)-1].shift(1)))
        data = data.groupby('Date_only').apply(replace_first_price)
        data1=data1[1:]
        data1["RS"]=data1["ui"]*(data1["ui"]-data1["ci"])+data1["di"]*(data1["di"]-data1["ci"])
        RS_var= data1["RS"].groupby(pd.Grouper(freq='D')).mean().dropna()
        Vc_and_Vo=data1[["oi", "ci"]].groupby(pd.Grouper(freq='D')).var().dropna()
        n=int(len(data1)/len(RS_var))
        k = 0.34/(1.34+(n+1)/(n-1))
        Yang_Zhang_RV=np.sqrt((1-k)*RS_var+Vc_and_Vo["oi"]+Vc_and_Vo["ci"]*k)
        Yang_Zhang_RV_df=pd.DataFrame(Yang_Zhang_RV)
        Yang_Zhang_RV_df.rename(columns={0: "Yang & Zhang RV proxy"},inplace=True)
        Multivariate_Yang_Zhang_RV.append(Yang_Zhang_RV_df)
    
    return Multivariate_Yang_Zhang_RV

In [21]:
#KLINE data
import pandas as pd
from binance import Client
from dotenv import dotenv_values
from datetime import datetime

config = dotenv_values('.env')
client = Client(config.get('KEY'), config.get('SECRET_KEY'))
TICKER = 'BTCUSDT'
start_date = datetime(2018, 1, 1)
end_date = datetime(2024, 3, 1)

start_date_str = start_date.strftime('%d %b, %Y')
end_date_str = end_date.strftime('%d %b, %Y')

klines = client.get_historical_klines(TICKER, client.KLINE_INTERVAL_1HOUR, start_date_str, end_date_str)
dataBTC = pd.DataFrame(
    data=[row[1:7] for row in klines],
    columns=['Open', 'High', 'Low', 'Close', 'Volume', 'Date'],
).set_index('Date')
dataBTC.index = pd.to_datetime(dataBTC.index, unit='ms')
dataBTC = dataBTC.sort_index()
dataBTC = dataBTC.apply(pd.to_numeric, axis=1)
dataBTC

,Open,High,Low,Close,Volume
Date,,,,,
2018-01-01 00:59:59.999,13715.65,13715.65,13400.01,13529.01,443.356199
2018-01-01 01:59:59.999,13528.99,13595.89,13155.38,13203.06,383.697006
2018-01-01 02:59:59.999,13203.00,13418.43,13200.00,13330.18,429.064572
2018-01-01 03:59:59.999,13330.26,13611.27,13290.00,13410.03,420.087030
2018-01-01 04:59:59.999,13434.98,13623.29,13322.15,13601.01,340.807329
...,...,...,...,...,...
2024-02-29 20:59:59.999,61599.99,62285.47,61521.73,61934.73,3755.220100
2024-02-29 21:59:59.999,61934.73,61999.75,60584.07,61374.94,4040.139080
2024-02-29 22:59:59.999,61374.95,61474.81,60672.82,61224.02,1906.566300


In [22]:
TICKER = 'ETHUSDT'
start_date = datetime(2018, 1, 1)
end_date = datetime(2024, 3, 1)

start_date_str = start_date.strftime('%d %b, %Y')
end_date_str = end_date.strftime('%d %b, %Y')

klines = client.get_historical_klines(TICKER, client.KLINE_INTERVAL_1HOUR, start_date_str, end_date_str)
dataETH = pd.DataFrame(
    data=[row[1:7] for row in klines],
    columns=['Open', 'High', 'Low', 'Close', 'Volume', 'Date'],
).set_index('Date')
dataETH.index = pd.to_datetime(dataETH.index, unit='ms')
dataETH = dataETH.sort_index()
dataETH = dataETH.apply(pd.to_numeric, axis=1)
dataETH

,Open,High,Low,Close,Volume
Date,,,,,
2018-01-01 00:59:59.999,733.01,734.52,720.03,727.62,2105.90100
2018-01-01 01:59:59.999,727.01,732.00,716.80,717.97,2305.97086
2018-01-01 02:59:59.999,717.67,725.75,717.59,724.05,2166.45725
2018-01-01 03:59:59.999,723.95,737.99,722.70,734.50,2160.90450
2018-01-01 04:59:59.999,734.99,744.98,730.01,744.82,2335.33705
...,...,...,...,...,...
2024-02-29 20:59:59.999,3403.27,3415.44,3393.61,3393.99,18045.00550
2024-02-29 21:59:59.999,3393.99,3398.63,3310.00,3347.74,40472.54590
2024-02-29 22:59:59.999,3347.73,3358.53,3300.00,3322.61,24605.94420


In [23]:
#calculating RV_BTC
RVBTC=Yang_Zhang_RV_own_data(dataBTC)
RVBTC['unique_id']=f'BTC'
RVBTC=RVBTC.rename(columns={'Yang & Zhang RV proxy':'BTC'})
RVBTC

,BTC,unique_id
Date,,
2018-01-01,0.014103,BTC
2018-01-02,0.018719,BTC
2018-01-03,0.014785,BTC
2018-01-04,0.013654,BTC
2018-01-05,0.016831,BTC
...,...,...
2024-02-26,0.004685,BTC
2024-02-27,0.007689,BTC
2024-02-28,0.015570,BTC


In [24]:
#calculating RV_ETH
RVETH=Yang_Zhang_RV_own_data(dataETH)
RVETH['unique_id']=f'ETH'

RVETH['BTC']=RVBTC['BTC']
RVETH

,Yang & Zhang RV proxy,unique_id,BTC
Date,,,
2018-01-01,0.011856,ETH,0.014103
2018-01-02,0.024648,ETH,0.018719
2018-01-03,0.019458,ETH,0.014785
2018-01-04,0.020923,ETH,0.013654
2018-01-05,0.016601,ETH,0.016831
...,...,...,...
2024-02-26,0.005743,ETH,0.004685
2024-02-27,0.006462,ETH,0.007689
2024-02-28,0.016756,ETH,0.015570


In [25]:
RVETH = RVETH.rename(columns={"Yang & Zhang RV proxy":"y"})
RVETH['ds']=RVETH.index.strftime('%Y-%m-%d')
RVETH['ds']=pd.to_datetime(RVETH['ds'])
RVETH=RVETH.dropna()
RVETH

,y,unique_id,BTC,ds
Date,,,,
2018-01-01,0.011856,ETH,0.014103,2018-01-01
2018-01-02,0.024648,ETH,0.018719,2018-01-02
2018-01-03,0.019458,ETH,0.014785,2018-01-03
2018-01-04,0.020923,ETH,0.013654,2018-01-04
2018-01-05,0.016601,ETH,0.016831,2018-01-05
...,...,...,...,...
2024-02-25,0.004872,ETH,0.002504,2024-02-25
2024-02-26,0.005743,ETH,0.004685,2024-02-26
2024-02-27,0.006462,ETH,0.007689,2024-02-27


In [26]:
# 2D-Wasserstein loss function
from ripser import Rips
import persim
def _divide_no_nan(a: torch.Tensor, b: torch.Tensor) -> torch.Tensor:
    """
    Auxiliary funtion to handle divide by 0
    """
    div = a / b
    div[div != div] = 0.0
    div[div == float("inf")] = 0.0
    return div

def _weighted_mean(losses, weights):
    """
    Compute weighted mean of losses per datapoint.
    """
    return _divide_no_nan(torch.sum(losses * weights), torch.sum(weights))

# %% ../../nbs/losses.pytorch.ipynb 7
class BasePointLoss(torch.nn.Module):
    """
    Base class for point loss functions.

    **Parameters:**
    `horizon_weight`: Tensor of size h, weight for each timestamp of the forecasting window. 
    `outputsize_multiplier`: Multiplier for the output size. 
    `output_names`: Names of the outputs. 
    """

    def __init__(self, horizon_weight, outputsize_multiplier, output_names):
        super(BasePointLoss, self).__init__()
        if horizon_weight is not None:
            horizon_weight = torch.Tensor(horizon_weight.flatten())
        self.horizon_weight = horizon_weight
        self.outputsize_multiplier = outputsize_multiplier
        self.output_names = output_names
        self.is_distribution_output = False

    def domain_map(self, y_hat: torch.Tensor):
        """
        Univariate loss operates in dimension [B,T,H]/[B,H]
        This changes the network's output from [B,H,1]->[B,H]
        """
        return y_hat.squeeze(-1)

    def _compute_weights(self, y, mask):
        """
        Compute final weights for each datapoint (based on all weights and all masks)
        Set horizon_weight to a ones[H] tensor if not set.
        If set, check that it has the same length as the horizon in x.
        """
        if mask is None:
            mask = torch.ones_like(y).to(y.device)

        if self.horizon_weight is None:
            self.horizon_weight = torch.ones(mask.shape[-1])
        else:
            assert mask.shape[-1] == len(
                self.horizon_weight
            ), "horizon_weight must have same length as Y"

        weights = self.horizon_weight.clone()
        weights = torch.ones_like(mask, device=mask.device) * weights.to(mask.device)
        return weights * mask


class MSE_2DWD(BasePointLoss):
    from ripser import Rips
    import persim

    def __init__(self, horizon_weight=None):
        super(MSE_2DWD, self).__init__(
            horizon_weight=horizon_weight, outputsize_multiplier=1, output_names=[""]
        )

    def __call__(
        self,
        y: torch.Tensor,
        y_hat: torch.Tensor,
        mask: Union[torch.Tensor, None] = None,
    ):
        """
        **Parameters:**
        `y`: tensor, Actual values.
        `y_hat`: tensor, Predicted values.
        `mask`: tensor, Specifies datapoints to consider in loss.

        **Returns:**
        `mse`: tensor (single value).
        """
        rips = Rips(maxdim = 2, verbose=False)
        n=2
        wasserstein_dists = np.zeros((n))

        for i in range(2):
          dgm1 = rips.fit_transform(y_hat[:,int(y_hat.shape[1]/2*i):int(y_hat.shape[1]/2*(i+1))].to('cpu').detach().numpy())
          dgm2 = rips.fit_transform(y[:,int(y_hat.shape[1]/2*i):int(y_hat.shape[1]/2*(i+1))].to('cpu').detach().numpy())
          wasserstein_dists[i] = persim.wasserstein(dgm1[0], dgm2[0], matching=False)

        losses = torch.sum((y - y_hat) ** 2)/(y_hat.shape[0]*y_hat.shape[1]) + torch.tensor((np.sum(wasserstein_dists))/n)
        return losses
     

In [27]:
#creating train dataset
train = RVETH.iloc[:int(len(RVETH)*0.7)]
validation_length=int(len(RVETH)*0.7)-int(len(RVETH)*0.5)
train

,y,unique_id,BTC,ds
Date,,,,
2018-01-01,0.011856,ETH,0.014103,2018-01-01
2018-01-02,0.024648,ETH,0.018719,2018-01-02
2018-01-03,0.019458,ETH,0.014785,2018-01-03
2018-01-04,0.020923,ETH,0.013654,2018-01-04
2018-01-05,0.016601,ETH,0.016831,2018-01-05
...,...,...,...,...
2022-04-21,0.006363,ETH,0.006513,2022-04-21
2022-04-22,0.006104,ETH,0.005723,2022-04-22
2022-04-23,0.003978,ETH,0.003116,2022-04-23


In [28]:
#Hyperparameter search
n_inputs = [3,5,10,15,21,42,84]
mlp_units = [[[712, 712], [712, 712]],[[512, 512], [512, 512]],[[250, 250], [250, 250]],[[100, 100], [100, 100]]]
epochs=[50,100,150,250,350,450,550,650,750]
learning_rate=[0.0005,0.0001,0.00005,0.00001]
num_lr_decays=[5,3,2,1]
scaler_type=["robust","standard",'minmax']
stack_types=[['identity','identity'],['identity','trend'],['seasonality','identity'],['seasonality','trend']]
n_harmonics=[0,0,1,1]
n_blocks=[[1, 1],[2, 2],[3, 3],[5, 5]]
n_polynomials=[0,1,0,1]
     

In [ ]:
#Hyperparameters tuning
import random
from ripser import Rips
import persim
for m in range (1):
  i=random.randrange(7)
  h=random.randrange(4)
  a=random.randrange(4)
  k=random.randrange(9)
  l=random.randrange(4)
  n=random.randrange(4)
  o=random.randrange(3)
  model = NBEATSx(h=14, input_size=n_inputs[i],
                loss=MSE_2DWD(),
                scaler_type=scaler_type[o],
                learning_rate=learning_rate[n],
                stack_types=stack_types[l],
                n_blocks=n_blocks[a],
                mlp_units=mlp_units[h],
                windows_batch_size=32,
                num_lr_decays=num_lr_decays[n],
                val_check_steps=500,
                hist_exog_list=['BTC'],
                n_harmonics=n_harmonics[l], n_polynomials=n_polynomials[l],
                max_steps=epochs[k],
                random_seed=random.randrange(129228148))
  fcst = NeuralForecast(models=[model],freq='D')
  forecasts = fcst.cross_validation(df=train,val_size=14,n_windows=None, test_size=validation_length-validation_length%14,step_size=1)
  forecasts = forecasts.dropna().drop_duplicates(subset='ds', keep='first')
  if "NBEATSx-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["NBEATSx"].values
  else:
    Y_hat=forecasts["NBEATSx-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | MLP units: {mlp_units[h]} | Scaler Type: {scaler_type[o]}")
  print(f"Stack Types: {stack_types[l]} | Learning Rate: {n} | Block: {n_blocks[a]}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAPE: {MAE}")

In [ ]:
#Second
for m in range (1):
  i=random.randrange(7)
  h=random.randrange(4)
  a=random.randrange(4)
  k=random.randrange(9)
  l=random.randrange(4)
  n=random.randrange(4)
  o=random.randrange(3)
  model = NBEATSx(h=14, input_size=n_inputs[i],
                loss=MSE_2DWD(),
                scaler_type=scaler_type[o],
                learning_rate=learning_rate[n],
                stack_types=stack_types[l],
                n_blocks=n_blocks[a],
                mlp_units=mlp_units[h],
                windows_batch_size=32,
                num_lr_decays=num_lr_decays[n],
                val_check_steps=500,
                hist_exog_list=['BTC'],
                n_harmonics=n_harmonics[l], n_polynomials=n_polynomials[l],
                max_steps=epochs[k],
                random_seed=random.randrange(129228148))
  fcst = NeuralForecast(models=[model],freq='D')
  forecasts = fcst.cross_validation(df=train,val_size=14,n_windows=None, test_size=validation_length-validation_length%14,step_size=1)
  forecasts = forecasts.dropna().drop_duplicates(subset='ds', keep='first')
  if "NBEATSx-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["NBEATSx"].values
  else:
    Y_hat=forecasts["NBEATSx-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | MLP units: {mlp_units[h]} | Scaler Type: {scaler_type[o]}")
  print(f"Stack Types: {stack_types[l]} | Learning Rate: {n} | Block: {n_blocks[a]}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAPE: {MAE}")

In [ ]:
#Third
for m in range (1):
  i=random.randrange(7)
  h=random.randrange(4)
  a=random.randrange(4)
  k=random.randrange(9)
  l=random.randrange(4)
  n=random.randrange(4)
  o=random.randrange(3)
  model = NBEATSx(h=14, input_size=n_inputs[i],
                loss=MSE_2DWD(),
                scaler_type=scaler_type[o],
                learning_rate=learning_rate[n],
                stack_types=stack_types[l],
                n_blocks=n_blocks[a],
                mlp_units=mlp_units[h],
                windows_batch_size=32,
                num_lr_decays=num_lr_decays[n],
                val_check_steps=500,
                hist_exog_list=['BTC'],
                n_harmonics=n_harmonics[l], n_polynomials=n_polynomials[l],
                max_steps=epochs[k],
                random_seed=random.randrange(129228148))
  fcst = NeuralForecast(models=[model],freq='D')
  forecasts = fcst.cross_validation(df=train,val_size=14,n_windows=None, test_size=validation_length-validation_length%14,step_size=1)
  forecasts = forecasts.dropna().drop_duplicates(subset='ds', keep='first')
  if "NBEATSx-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["NBEATSx"].values
  else:
    Y_hat=forecasts["NBEATSx-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | MLP units: {mlp_units[h]} | Scaler Type: {scaler_type[o]}")
  print(f"Stack Types: {stack_types[l]} | Learning Rate: {n} | Block: {n_blocks[a]}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAPE: {MAE}")

In [ ]:
#Fourth
for m in range (1):
  i=random.randrange(7)
  h=random.randrange(4)
  a=random.randrange(4)
  k=random.randrange(9)
  l=random.randrange(4)
  n=random.randrange(4)
  o=random.randrange(3)
  model = NBEATSx(h=14, input_size=n_inputs[i],
                loss=MSE_2DWD(),
                scaler_type=scaler_type[o],
                learning_rate=learning_rate[n],
                stack_types=stack_types[l],
                n_blocks=n_blocks[a],
                mlp_units=mlp_units[h],
                windows_batch_size=32,
                num_lr_decays=num_lr_decays[n],
                val_check_steps=500,
                hist_exog_list=['BTC'],
                n_harmonics=n_harmonics[l], n_polynomials=n_polynomials[l],
                max_steps=epochs[k],
                random_seed=random.randrange(129228148))
  fcst = NeuralForecast(models=[model],freq='D')
  forecasts = fcst.cross_validation(df=train,val_size=14,n_windows=None, test_size=validation_length-validation_length%14,step_size=1)
  forecasts = forecasts.dropna().drop_duplicates(subset='ds', keep='first')
  if "NBEATSx-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["NBEATSx"].values
  else:
    Y_hat=forecasts["NBEATSx-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | MLP units: {mlp_units[h]} | Scaler Type: {scaler_type[o]}")
  print(f"Stack Types: {stack_types[l]} | Learning Rate: {n} | Block: {n_blocks[a]}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAPE: {MAE}")

In [ ]:
#Fifth
for m in range (1):
  i=random.randrange(7)
  h=random.randrange(4)
  a=random.randrange(4)
  k=random.randrange(9)
  l=random.randrange(4)
  n=random.randrange(4)
  o=random.randrange(3)
  model = NBEATSx(h=14, input_size=n_inputs[i],
                loss=MSE_2DWD(),
                scaler_type=scaler_type[o],
                learning_rate=learning_rate[n],
                stack_types=stack_types[l],
                n_blocks=n_blocks[a],
                mlp_units=mlp_units[h],
                windows_batch_size=32,
                num_lr_decays=num_lr_decays[n],
                val_check_steps=500,
                hist_exog_list=['BTC'],
                n_harmonics=n_harmonics[l], n_polynomials=n_polynomials[l],
                max_steps=epochs[k],
                random_seed=random.randrange(129228148))
  fcst = NeuralForecast(models=[model],freq='D')
  forecasts = fcst.cross_validation(df=train,val_size=14,n_windows=None, test_size=validation_length-validation_length%14,step_size=1)
  forecasts = forecasts.dropna().drop_duplicates(subset='ds', keep='first')
  if "NBEATSx-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["NBEATSx"].values
  else:
    Y_hat=forecasts["NBEATSx-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | MLP units: {mlp_units[h]} | Scaler Type: {scaler_type[o]}")
  print(f"Stack Types: {stack_types[l]} | Learning Rate: {n} | Block: {n_blocks[a]}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAPE: {MAE}")

In [ ]:
#Random seed tuning
for m in range (5):
  randomseed=random.randrange(121384641)
  model = NBEATSx(h=14, input_size=84,
                loss=MSE_2DWD(),
                scaler_type='robust',
                learning_rate=learning_rate[0],
                stack_types=['identity', 'trend'],
                n_blocks=[5,5],
                mlp_units=[[512, 512], [512, 512]],
                windows_batch_size=32,
                num_lr_decays=num_lr_decays[0],
                val_check_steps=500,
                hist_exog_list=['BTC'],
                n_harmonics=n_harmonics[1], n_polynomials=n_polynomials[1],
                max_steps=100,
                random_seed=randomseed)
  fcst = NeuralForecast(models=[model],freq='D')
  forecasts = fcst.cross_validation(df=train,val_size=14,n_windows=None, test_size=validation_length-validation_length%14,step_size=1)
  forecasts = forecasts.dropna().drop_duplicates(subset='ds', keep='first')
  if "NBEATSx-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["NBEATSx"].values
  else:
    Y_hat=forecasts["NBEATSx-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  print(f"Random Seed: {randomseed}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAPE: {MAE}")

In [ ]:
#second
for m in range (5):
  randomseed=random.randrange(121384641)
  model = NBEATSx(h=14, input_size=84,
                loss=MSE_2DWD(),
                scaler_type='robust',
                learning_rate=learning_rate[0],
                stack_types=['identity', 'trend'],
                n_blocks=[5,5],
                mlp_units=[[512, 512], [512, 512]],
                windows_batch_size=32,
                num_lr_decays=num_lr_decays[0],
                val_check_steps=500,
                hist_exog_list=['BTC'],
                n_harmonics=n_harmonics[1], n_polynomials=n_polynomials[1],
                max_steps=100,
                random_seed=randomseed)
  fcst = NeuralForecast(models=[model],freq='D')
  forecasts = fcst.cross_validation(df=train,val_size=14,n_windows=None, test_size=validation_length-validation_length%14,step_size=1)
  forecasts = forecasts.dropna().drop_duplicates(subset='ds', keep='first')
  if "NBEATSx-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["NBEATSx"].values
  else:
    Y_hat=forecasts["NBEATSx-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  print(f"Random Seed: {randomseed}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAPE: {MAE}")

In [ ]:
#Third
for m in range (5):
  randomseed=random.randrange(121384641)
  model = NBEATSx(h=14, input_size=84,
                loss=MSE_2DWD(),
                scaler_type='robust',
                learning_rate=learning_rate[0],
                stack_types=['identity', 'trend'],
                n_blocks=[5,5],
                mlp_units=[[512, 512], [512, 512]],
                windows_batch_size=32,
                num_lr_decays=num_lr_decays[0],
                val_check_steps=500,
                hist_exog_list=['BTC'],
                n_harmonics=n_harmonics[1], n_polynomials=n_polynomials[1],
                max_steps=100,
                random_seed=randomseed)
  fcst = NeuralForecast(models=[model],freq='D')
  forecasts = fcst.cross_validation(df=train,val_size=14,n_windows=None, test_size=validation_length-validation_length%14,step_size=1)
  forecasts = forecasts.dropna().drop_duplicates(subset='ds', keep='first')
  if "NBEATSx-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["NBEATSx"].values
  else:
    Y_hat=forecasts["NBEATSx-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  print(f"Random Seed: {randomseed}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAPE: {MAE}")

In [29]:
model = NBEATSx(h=14, input_size=84,
                loss=MSE_2DWD(),
                scaler_type='robust',
                learning_rate=learning_rate[0],
                stack_types=['identity', 'trend'],
                n_blocks=[5,5],
                mlp_units=[[512, 512], [512, 512]],
                windows_batch_size=32,
                num_lr_decays=num_lr_decays[0],
                val_check_steps=500,
                hist_exog_list=['BTC'],
                n_harmonics=n_harmonics[1], n_polynomials=n_polynomials[1],
                max_steps=100,
                random_seed=53011437)
fcst = NeuralForecast(models=[model],freq='D')

forecasts = fcst.cross_validation(df=train,val_size=14, n_windows=None, test_size=validation_length-validation_length%14,step_size=1)
forecasts = forecasts.dropna().drop_duplicates(subset='ds', keep='first')
if "NBEATSx-median" not in list(forecasts.columns.values):
  Y_hat =forecasts["NBEATSx"].values
else:
  Y_hat =forecasts["NBEATSx-median"].values
Y_true=forecasts["y"].values
RMSE_testing=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
QLIKE_testing=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
MAE_testing=np.sum(abs(Y_true-Y_hat))/len(Y_true)
print(f"RMSE: {RMSE_testing} | QLIKE: {QLIKE_testing} | MAE: {MAE_testing}")

Seed set to 53011437


Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

Training: |                                               | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Predicting: |                                             | 0/? [00:00<?, ?it/s]

RMSE: 0.006660701069529581 | QLIKE: 0.11360665986172401 | MAE: 0.004245498396394229


In [30]:
#Testing
test_length=int(len(RVETH)*0.3)

In [31]:
model = NBEATSx(h=14, input_size=84,
                loss=MSE_2DWD(),
                scaler_type='robust',
                learning_rate=learning_rate[0],
                stack_types=['identity', 'trend'],
                n_blocks=[5,5],
                mlp_units=[[512, 512], [512, 512]],
                windows_batch_size=32,
                num_lr_decays=num_lr_decays[0],
                val_check_steps=500,
                hist_exog_list=['BTC'],
                n_harmonics=n_harmonics[1], n_polynomials=n_polynomials[1],
                max_steps=100,
                random_seed=53011437)
fcst = NeuralForecast(models=[model],freq='D')

forecasts = fcst.cross_validation(df=RVETH,val_size=14, n_windows=None, test_size=test_length-test_length%14,step_size=1)
forecasts = forecasts.dropna().drop_duplicates(subset='ds', keep='first')
if "NBEATSx-median" not in list(forecasts.columns.values):
  Y_hat =forecasts["NBEATSx"].values
else:
  Y_hat =forecasts["NBEATSx-median"].values
Y_true=forecasts["y"].values
RMSE_testing=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
QLIKE_testing=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
MAE_testing=np.sum(abs(Y_true-Y_hat))/len(Y_true)
print(f"RMSE: {RMSE_testing} | QLIKE: {QLIKE_testing} | MAE: {MAE_testing}")

Seed set to 53011437


Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

Training: |                                               | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Predicting: |                                             | 0/? [00:00<?, ?it/s]

RMSE: 0.0035208286159301543 | QLIKE: 0.12023179299185455 | MAE: 0.0024182570662525786


In [36]:
import plotly.express as px
import pandas as pd
import numpy as np
fig = px.line(forecasts, x='ds', y=forecasts.columns[3:5], color='variable', title='H=14 N-BEATSx prediction')

fig.show()

In [38]:
forecasts=forecasts.rename(columns={'y':'Real_value','BTC':'external_BTC'})
forecasts

,unique_id,ds,cutoff,NBEATSx,Real_value,external_BTC
0,ETH,2022-04-29,2022-04-28,0.007338,0.006459,0.005412
1,ETH,2022-04-30,2022-04-28,0.006799,0.005403,0.003551
2,ETH,2022-05-01,2022-04-28,0.006942,0.006518,0.005401
3,ETH,2022-05-02,2022-04-28,0.006955,0.006823,0.006042
4,ETH,2022-05-03,2022-04-28,0.007470,0.005157,0.004382
...,...,...,...,...,...,...
9169,ETH,2024-02-25,2024-02-11,0.003961,0.004872,0.002504
9183,ETH,2024-02-26,2024-02-12,0.004257,0.005743,0.004685
9197,ETH,2024-02-27,2024-02-13,0.005152,0.006462,0.007689
9211,ETH,2024-02-28,2024-02-14,0.004550,0.016756,0.015570


In [39]:
forecasts.to_csv('ETH_H=14_NBEATSx_modify.csv')